In [1]:
from selenium import webdriver  
from bs4 import BeautifulSoup
import requests

booking_link = "https://www.booking.com"
region_link = "https://www.booking.com/searchresults.html?dest_id=3131&dest_type=region&ss=Alberta%2C%20Canada&offset=0"

In [2]:
def review_url(hotel_link):
    result = requests.get(hotel_link)
    content = result.content
    soup = BeautifulSoup(content, "html.parser")
    rvbtns = soup.find_all("a", attrs={"class": "show_all_reviews_btn"})
    reviews_link = ""
    if len(rvbtns) == 1:
        reviews_link = rvbtns[0].get("href").strip()
    return reviews_link

def review_rp(hotel_link):
    return hotel_link.split("?")[0].replace("/hotel/ca/", "/reviews/ca/hotel/")


def url1page(region_link, booking_link):
    result = requests.get(region_link)
    content = result.content
    soup = BeautifulSoup(content, "html.parser")
    hotel_links = []
    hotel_names = []
    hlinks = soup.find_all("div", attrs={"class": 'sr_item sr_item_new sr_item_default sr_property_block sr_item_no_dates '})
    for hlink in hlinks:
        hname = hlink.find_all("span", attrs={"class": "sr-hotel__name"})[0].get_text().strip()
        hurl = hlink.find_all("a", attrs={"class": "hotel_name_link url"})[0].get("href").strip()
        revs = hlink.find_all("span", attrs={"class": 'review-score-widget__subtext'})
        if len(revs) == 1:
            no_revs = int(revs[0].get_text().strip().split(" ")[0].replace(",",""))
            if no_revs >= 50:
#                 hotel_links.append((hname, no_revs, hurl))
                rvurl = review_rp(hurl)
                hurl = booking_link + hurl
                rvurl = booking_link + rvurl
                hotel_links.append((hname, no_revs, hurl, rvurl))
        
    nxtbtns = soup.find_all("a", attrs={"title": "Next page"})
    nxt_link = ""
    if len(nxtbtns) == 1:
        nxt_link = nxtbtns[0].get("href")
    return hotel_links, nxt_link
        
hotel_links, nxt_link = url1page(region_link, booking_link)
hotel_links[:5]

[('Lake Louise Inn',
  3101,
  'https://www.booking.com/hotel/ca/lake-louise-inn.html?srpvid=769f6b0ea63f01e4&srepoch=1537456414&room1=A,A&hpos=1&hapos=1&dest_type=region&dest_id=3131&from=searchresults\n#hotelTmpl',
  'https://www.booking.com/reviews/ca/hotel/lake-louise-inn.html'),
 ('Solara Resort - Bellstar Hotels & Resorts',
  1136,
  'https://www.booking.com/hotel/ca/solara-resort.html?srpvid=769f6b0ea63f01e4&srepoch=1537456414&room1=A,A&hpos=2&hapos=2&dest_type=region&dest_id=3131&from=searchresults\n#hotelTmpl',
  'https://www.booking.com/reviews/ca/hotel/solara-resort.html'),
 ('Elk + Avenue Hotel',
  1434,
  'https://www.booking.com/hotel/ca/banff-international.html?srpvid=769f6b0ea63f01e4&srepoch=1537456414&room1=A,A&hpos=3&hapos=3&dest_type=region&dest_id=3131&from=searchresults\n#hotelTmpl',
  'https://www.booking.com/reviews/ca/hotel/banff-international.html'),
 ('Basecamp Resorts',
  463,
  'https://www.booking.com/hotel/ca/base-camp-resorts.html?srpvid=769f6b0ea63f01e4&

In [3]:
def hotel_urls(region_link, booking_link, max_page=5):
    c = 0
    list_hotel, nxt_link = url1page(region_link, booking_link)
    while len(nxt_link)!=0:
        print(nxt_link)
        if max_page > 0:
            c += 1
            if c == max_page:
                break
        hotels, nxt_link = url1page(nxt_link, booking_link)
        list_hotel.extend(hotels)
    list_hotel.sort(key=lambda x: x[1], reverse=True)
    return list_hotel

hotel_links = hotel_urls(region_link, booking_link)
hotel_links[:5]

https://www.booking.com/searchresults.html?dest_id=3131&dest_type=region&ss=Alberta%2C%20Canada&offset=15
https://www.booking.com/searchresults.html?dest_id=3131&dest_type=region&ss=Alberta%2C%20Canada&offset=30
https://www.booking.com/searchresults.html?dest_id=3131&dest_type=region&ss=Alberta%2C%20Canada&offset=45
https://www.booking.com/searchresults.html?dest_id=3131&dest_type=region&ss=Alberta%2C%20Canada&offset=60
https://www.booking.com/searchresults.html?dest_id=3131&dest_type=region&ss=Alberta%2C%20Canada&offset=75


[('Lake Louise Inn',
  3101,
  'https://www.booking.com/hotel/ca/lake-louise-inn.html?srpvid=672c6b102871009c&srepoch=1537456417&room1=A,A&hpos=1&hapos=1&dest_type=region&dest_id=3131&from=searchresults\n#hotelTmpl',
  'https://www.booking.com/reviews/ca/hotel/lake-louise-inn.html'),
 ('Wingate by Wyndham Calgary Airport',
  2522,
  'https://www.booking.com/hotel/ca/wingate-by-wyndham-calgary-airport.html?srpvid=70f76b118e2d0277&srepoch=1537456418&room1=A,A&hpos=15&hapos=30&dest_type=region&dest_id=3131&from=searchresults\n#hotelTmpl',
  'https://www.booking.com/reviews/ca/hotel/wingate-by-wyndham-calgary-airport.html'),
 ('Wyndham Garden Calgary Airport',
  2203,
  'https://www.booking.com/hotel/ca/wyndham-garden-calgary-airport.html?srpvid=c9646b1206d801d7&srepoch=1537456420&room1=A,A&hpos=12&hapos=42&dest_type=region&dest_id=3131&from=searchresults\n#hotelTmpl',
  'https://www.booking.com/reviews/ca/hotel/wyndham-garden-calgary-airport.html'),
 ('Maligne Lodge',
  2183,
  'https://w

In [4]:
import time

def rv1page(rv_link):
    result = requests.get(rv_link)
    content = result.content
    soup = BeautifulSoup(content, "html.parser")
    rvconts = soup.find_all("div", attrs={"class": "review_item_review_content"})
    nxtpage = soup.find_all("a", attrs={"id": "review_next_page_link"})
    nxt_link = ""
    if len(nxtpage) != 0:
        nxt_link = nxtpage[0].get("href").strip()
    pos_rvs = []
    neg_rvs = []
    for rvcont in rvconts:
        pos = rvcont.find_all("p", attrs={"class": "review_pos "})
        if len(pos) == 1:
            pos_rv = pos[0].find_all("span", attrs={"itemprop": "reviewBody"})[0].text.strip()
            if len(pos_rv.split()) >= 3:
                pos_rvs.append(pos_rv)

        neg = rvcont.find_all("p", attrs={"class": "review_neg "})
        if len(neg) == 1:
            neg_rv = neg[0].find_all("span", attrs={"itemprop": "reviewBody"})[0].text.strip()
            if len(neg_rv.split()) >= 3:
                neg_rvs.append(neg_rv)
    return pos_rvs, neg_rvs, nxt_link

def rv1hotel(rv_link, booking_link):
    start = time.time()
    pos_rvs, neg_rvs, nxt = rv1page(rv_link)
    while len(nxt) != 0:
        nxt = booking_link + nxt
        pos, neg, nxt = rv1page(nxt)
        pos_rvs.extend(pos)
        neg_rvs.extend(neg)
    now = time.time() - start
    print("\t+ Extracted %d negative and %d positive reviews in %.4f(s); speed: %.2f(reviews/s)." % (len(neg_rvs), len(pos_rvs), now, (len(neg_rvs) + len(pos_rvs))/now))
    return pos_rvs, neg_rvs

rv_link = hotel_links[-1][-1]
pos_rvs, neg_rvs = rv1hotel(rv_link, booking_link)

	+ Extracted 131 negative and 137 positive reviews in 5.5079(s); speed: 48.66(reviews/s).


In [5]:
def rvnhotel(hotel_links, booking_link, n=5):
    pos_data = []
    neg_data = []
    start = time.time()
    hotel_links = hotel_links[:n]
    for hotel_link in hotel_links:
        print("- Scraping hotel '%s':" % hotel_link[0])
        rv_link = hotel_link[-1]
        pos_rvs, neg_rvs = rv1hotel(rv_link, booking_link)
        pos_data.extend(pos_rvs)
        neg_data.extend(neg_rvs)
    now = time.time() - start
    print("Scraping all %d hotels in %.4f(s); average speed %.2f(reviews/s)"%((len(hotel_links), now, (len(neg_data) + len(pos_data))/now)))
    return pos_data, neg_data

pos_data, neg_data = rvnhotel(hotel_links, booking_link, n=5)

- Scraping hotel 'Lake Louise Inn':
	+ Extracted 995 negative and 1052 positive reviews in 50.8418(s); speed: 40.26(reviews/s).
- Scraping hotel 'Wingate by Wyndham Calgary Airport':
	+ Extracted 689 negative and 1017 positive reviews in 40.3531(s); speed: 42.28(reviews/s).
- Scraping hotel 'Wyndham Garden Calgary Airport':
	+ Extracted 664 negative and 962 positive reviews in 39.8075(s); speed: 40.85(reviews/s).
- Scraping hotel 'Maligne Lodge':
	+ Extracted 802 negative and 824 positive reviews in 44.8815(s); speed: 36.23(reviews/s).
- Scraping hotel 'Days Inn by Wyndham Calgary South':
	+ Extracted 942 negative and 834 positive reviews in 38.3798(s); speed: 46.27(reviews/s).
Scraping all 5 hotels in 214.2741(s); average speed 40.98(reviews/s)


In [6]:
print(len(pos_data))
print(len(neg_data))

4689
4092


In [7]:
pos_data

['Nice loft accommodations and facilities (restaurant & pool)',
 'Very convenient, free parking, can walk to dining options in town and across the street.',
 'A well-presented hotel both exterior and interior.  The room was very comfortable with all the amenities needed.  The indoor swimming pool and whirl pools were a very welcome pleasure after some hard walking around the Lake.\rThe hotel is conveniently situated for walking to the small local shopping mall/restaurants/coffee shops.',
 'Location for Lake Louise was very good.',
 'I like the kitchen of the property, It just provide everything that we want.',
 'I liked the location but that is about it. Staff were amazing too! fantastic customer service!',
 'Location was great',
 'We did not eat ate this facility.  I chose not to spend any additional money with them.',
 'We did enjoy the restaurant on site Timberwolf. Pizza good but skimpy with toppings. Loved the location.',
 'The room was comfortable and the restaurants on site were

In [8]:
neg_data

['Very expensive ($500 USD per night) yet hotel is not close to Lake Louise (still need to walk a fair distance or alternatively drive a few miles to overflow parking and wait in crowded lines for buses)',
 'We had a room with no windows to the outside.  I had no idea that would be the case.  We had a dark room overlooking the pool and it felt crowded and claustrophobic.  Tour busses stay here and when they unload - watch out - the lobby and restaurnats get crowded.',
 'Very dated decor in the rooms. It felt very dark and a throw back to the 80s. Food offering for breakfast was very disappointing.\nThe area had a power cut over night whilst we were staying. The next day was chaos. There was a wait on for breakfast and the staff were clearly struggling to deal with it.',
 'The Main problem of this inn was they just held my deposit in the credit card for several days which made me quite difficult for a trip to use the credit card.',
 'Luckily i chose to be on the "ground floor" as the bu

In [31]:
region_link = "https://www.booking.com/searchresults.html?label=gen173nr-1DCAQoggJCDXNlYXJjaF9jYW5hZGFIM1gEaCeIAQGYATG4AQfIAQ_YAQPoAQH4AQOSAgF5qAID&sid=b5e75395288459d86dce8ee5ff5808ff&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.html%3Flabel%3Dgen173nr-1DCAQoggJCDXNlYXJjaF9jYW5hZGFIM1gEaCeIAQGYATG4AQfIAQ_YAQPoAQH4AQOSAgF5qAID%3Bsid%3Db5e75395288459d86dce8ee5ff5808ff%3Bclass_interval%3D1%3Bdest_id%3D38%3Bdest_type%3Dcountry%3Bdtdisc%3D0%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsrpvid%3D36a076fc587e024b%3Bss%3DCanada%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%26%3B&ss=Ontario%2C+Canada&ssne=Canada&ssne_untouched=Canada&checkin_month=&checkin_monthday=&checkin_year=&checkout_month=&checkout_monthday=&checkout_year=&no_rooms=1&group_adults=2&group_children=0&b_h4u_keep_filters=&from_sf=1&ss_raw=ontario&ac_position=1&ac_langcode=en&dest_id=659&dest_type=region&place_id_lat=44.13311&place_id_lon=-79.535998&search_pageview_id=36a076fc587e024b&search_selected=true&search_pageview_id=36a076fc587e024b&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0"
result = requests.get(region_link)
content = result.content
soup = BeautifulSoup(content, "html.parser")

In [32]:
hotel_links = []
hotel_names = []
hlinks = soup.find_all("div", attrs={"class": 'sr_item sr_item_new sr_item_default sr_property_block sr_item_no_dates '})
for hlink in hlinks:
    hname = hlink.find_all("span", attrs={"class": "sr-hotel__name"})[0].get_text().strip()
    hurl = hlink.find_all("a", attrs={"class": "hotel_name_link url"})[0].get("href").strip()
    revs = hlink.find_all("span", attrs={"class": 'review-score-widget__subtext'})
    if len(revs) == 1:
        no_revs = int(revs[0].get_text().strip().split(" ")[0].replace(",",""))
        if no_revs >= 50:
#                 hotel_links.append((hname, no_revs, hurl))
            rvurl = review_rp(hurl)
            hurl = booking_link + hurl
            rvurl = booking_link + rvurl
            hotel_links.append((hname, no_revs, hurl, rvurl))

nxtbtns = soup.find_all("a", attrs={"title": "Next page"})
nxt_link = ""
if len(nxtbtns) == 1:
    nxt_link = nxtbtns[0].get("href")

In [33]:
nxtbtns

[<a class="bui-pagination__link paging-next ga_sr_gotopage_2_211" data-page-next="" href="https://www.booking.com/searchresults.html?dest_id=659&amp;dest_type=region&amp;ss=Ontario%2C%2BCanada&amp;offset=15" title="Next page">
 <svg class="bk-icon -iconset-navarrow_right bui-pagination__icon" height="18" role="presentation" viewbox="0 0 128 128" width="18"><path d="M54.3 96a4 4 0 0 1-2.8-6.8L76.7 64 51.5 38.8a4 4 0 0 1 5.7-5.6L88 64 57.2 94.8a4 4 0 0 1-2.9 1.2z"></path></svg>
 </a>]

In [30]:
soup

<!DOCTYPE html>

<!--
You know you could be getting paid to poke around in our code?
We're hiring designers and developers to work in Amsterdam:
http://www.workingatbooking.com/
-->
<!-- wdot-802 -->
<html class="noJS b_bot b_bot supports_fontface " lang="en-us" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# booking_com: http://ogp.me/ns/fb/booking_com#">
<head profile="http://a9.com/-/spec/opensearch/1.1/">
<meta content="origin" name="referrer"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type">
<script> function b_cors_check(supported) { var value = supported ? 1 : 0; if (!/(^|;)\s*cors_js=/.test(document.cookie)) { var d = new Date(); d.setTime(d.getTime() + 60 * 60 * 24 * 365 * 1000); var cookieDomain = '.booking.com' || '.booking.com'; document.cookie = 'cors_js=' + value +'; domain=' + cookieDomain + '; path=/; expires=' + d.toGMTString(); } if (!value) { location.reload(); } } </script>
<script async="" crossorigin="" onerror="b_cors_check(false)" src=

In [ ]:
//*[@id="hotellist_inner"]